In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sqlite3
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nyse/prices-split-adjusted.csv
/kaggle/input/nyse/fundamentals.csv
/kaggle/input/nyse/prices.csv
/kaggle/input/nyse/securities.csv


In [2]:
#Formatting the output of a SELECT query
def format_table(column_names, rows, rows_limit = 10, cols_limit = 7, spacing = 15):
    if len(column_names) > cols_limit:
        column_names = column_names[:cols_limit] + ['...']
        rows = [row[:cols_limit] for row in rows]
    print([f"{cname : ^{spacing}}" for cname in column_names])
    print('-' * (spacing + 5) * len(column_names))
    rows_num = 0
    for row in rows:
        print([f"{entry if entry is not None else 'NULL' : ^{spacing}}" for entry in row])
        rows_num += 1
        if rows_num >= rows_limit: break;
    print('\n')

<h2>About the Dataset</h2>

The dataset comprises of:
<ul>
    <li>prices.csv: raw, as-is daily prices. Most of data spans from 2010 to the end 2016, for companies new on stock market date range is shorter. There have been approx. 140 stock splits in that time, this set doesn't account for that.</li>
    <li>prices-split-adjusted.csv: same as prices, but there have been added adjustments for splits.</li>
    <li>securities.csv: general description of each company with division on sectors.</li>
    <li>fundamentals.csv: metrics extracted from annual SEC 10K fillings (2012-2016).</li>
</ul>
Prices were fetched from Yahoo Finance, fundamentals are from Nasdaq Financials, extended by some fields from EDGAR SEC databases.

<h2>DATABASE CREATION</h2>

We load the dataset and create a database with a table for each CSV file:

In [3]:
fundamentals_df = pd.read_csv('/kaggle/input/nyse/fundamentals.csv')
prices_df = pd.read_csv('/kaggle/input/nyse/prices.csv')
prices_split_adjusted_df = pd.read_csv('/kaggle/input/nyse/prices-split-adjusted.csv')
securities_df = pd.read_csv('/kaggle/input/nyse/securities.csv')

In [4]:
conn = sqlite3.connect('stock_data')
fundamentals_df.to_sql('fundamentals', conn, if_exists = 'replace',  index = False)
prices_df.to_sql('prices', conn, if_exists = 'replace',  index = False)
prices_split_adjusted_df.to_sql('prices_adj', conn, if_exists = 'replace',  index = False)
securities_df.to_sql('securities', conn, if_exists = 'replace',  index = False)

505

We can quickly visualize the first rows of each table:

In [5]:
TABLES = ['fundamentals', 'prices', 'prices_adj', 'securities']
for table in TABLES:
    cur = conn.cursor()
    sql_view = f"SELECT * FROM {table} LIMIT 5;"
    cur.execute(sql_view)
    rows = cur.fetchall()
    cnames = list(map(lambda x : x[0],  cur.description)) 
    cur.close()
    print(' ' * 40 + table.upper())
    format_table(cnames, rows)

                                        FUNDAMENTALS
['  Unnamed: 0   ', ' Ticker Symbol ', ' Period Ending ', 'Accounts Payable', 'Accounts Receivable', "Add'l income/expense items", ' After Tax ROE ', '      ...      ']
----------------------------------------------------------------------------------------------------------------------------------------------------------------
['       0       ', '      AAL      ', '  2012-12-31   ', ' 3068000000.0  ', ' -222000000.0  ', ' -1961000000.0 ', '     23.0      ']
['       1       ', '      AAL      ', '  2013-12-31   ', ' 4975000000.0  ', '  -93000000.0  ', ' -2723000000.0 ', '     67.0      ']
['       2       ', '      AAL      ', '  2014-12-31   ', ' 4668000000.0  ', ' -160000000.0  ', ' -150000000.0  ', '     143.0     ']
['       3       ', '      AAL      ', '  2015-12-31   ', ' 5102000000.0  ', '  352000000.0  ', ' -708000000.0  ', '     135.0     ']
['       4       ', '      AAP      ', '  2012-12-29   ', ' 2409453000.0  ', '  -

<h2>UNIFORM THE DATA</h2>

While the end data of data collection is the same for all symbols, the starting date differs, we look at the distribution of starting dates to check if there's relevant mode we can use:

In [6]:
sql = """
WITH symbols_dates AS(
SELECT symbol, MIN(date) AS min_date,
MAX(date) AS max_date
FROM prices
GROUP BY symbol)
SELECT  min_date as unique_min_date,
COUNT(min_date) as num_date
FROM symbols_dates
GROUP BY unique_min_date
ORDER BY num_date DESC
"""

cur = conn.cursor()
cur.execute(sql)
rows = cur.fetchall()
cnames = list(map(lambda x : x[0], cur.description))
cur.close()
format_table(cnames, rows)

['unique_min_date', '   num_date    ']
----------------------------------------
['  2010-01-04   ', '      466      ']
['  2013-01-02   ', '      13       ']
['  2015-01-02   ', '       4       ']
['  2015-07-06   ', '       2       ']
['  2013-06-19   ', '       2       ']
['2016-07-05 00:00:00', '       1       ']
['2016-01-05 00:00:00', '       1       ']
['  2015-11-16   ', '       1       ']
['  2015-10-19   ', '       1       ']
['  2015-06-24   ', '       1       ']




Over all the 493 symbols, only 26 has a starting date that's not 2010/01/04, so we decide to discard them; we first select the symbols with longest data collection:

In [7]:
sql = """
CREATE VIEW IF NOT EXISTS symbols_uniformed AS
SELECT * FROM (
SELECT 
symbol,
MIN(date) AS min_date,
MAX(date) AS max_date
FROM prices
GROUP BY symbol)
WHERE min_date = "2010-01-04"
;
"""
cur = conn.cursor()
cur.execute(sql)
cur.close()

and then create a table for all data from the prices table selecting only these symbols:

In [8]:
sql = """
DROP TABLE IF EXISTS price_unifored;
CREATE TABLE IF NOT EXISTS price_uniformed AS
SELECT * 
FROM prices
WHERE prices.symbol IN (SELECT symbol FROM symbols_uniformed)
;
"""
cur = conn.cursor()
cur.executescript(sql)
cur.close()

<h2>SECTORS SHARPE RATIO</h2>

Sharpe ratio is an indicator to evaluate a portfolio return in relation to the associated risk, the higher the Sharpe ratio the more confident we can be about the portfolio (albeit it's not tipically a sufficient indicator alone).</br>
The aim of this section is then to evaluate the most promising sectors; we therefore need to associate each stock with the sector it belongs to: the latter is defined in the securities table, we first rename the columns we are interested in:

In [9]:
sql_rename = """
ALTER TABLE securities RENAME COLUMN "Ticker symbol" TO symbol;
ALTER TABLE securities RENAME COLUMN "GICS Sector" TO sector;
"""
cur = conn.cursor()
cur.executescript(sql_rename)
conn.commit()
cur.close()

We can now join them with the stock data in a new view, we also add a column with the return values which we'll need to compute the Sharpe ratio:

In [10]:
def get_return(x):
    try:
        return np.log(x) + 1
    except:
        return x

conn.create_function('GET_RETURN', 1, get_return)

sql_symb_sec = """
DROP VIEW IF EXISTS symbol_sector;
CREATE VIEW IF NOT EXISTS symbol_sector AS
SELECT 
prices.*,
securities.sector,
get_return(close / (LAG(close, 1, NULL) OVER (PARTITION BY prices.symbol ORDER BY date ASC))) AS return
FROM prices INNER JOIN securities ON prices.symbol = securities.symbol;
"""

cur = conn.cursor()
cur.executescript(sql_symb_sec)
cur.close()

With the aid of user defined function we can write a query to return the Sharpe ration of each sector:

In [11]:
def get_sqrt(x):
    try:
        return np.sqrt(x)
    except:
        return x

conn.create_function('SQRT', 1, get_sqrt)

def get_return(risk_free):
    
    params_dict = {'risk_free' : risk_free}
    
    sql_rtrn = """
    WITH return_avg AS
    (SELECT sector, AVG(return) AS return_mean FROM symbol_sector GROUP BY sector),
    sector_mean_std AS
    (SELECT symbol_sector.sector, return_mean, SQRT(SUM((return - return_mean)*(return - return_mean))/(COUNT(return) - 1)) AS return_std 
    FROM symbol_sector JOIN return_avg ON symbol_sector.sector = return_avg.sector GROUP BY symbol_sector.sector)
    SELECT sector, (((return_mean - 1) * 252) - :risk_free) / return_std AS sharpe_ratio  FROM sector_mean_std ORDER BY sharpe_ratio DESC;
    """
    cur = conn.cursor()
    cur.execute(sql_rtrn, params_dict)
    rows = cur.fetchall()
    cnames = list(map(lambda x : x[0], cur.description))
    cur.close()
    format_table(cnames, rows, rows_limit = 15)

get_return(0.032)

['    sector     ', ' sharpe_ratio  ']
----------------------------------------
['  Real Estate  ', '4.250370932863932']
['  Health Care  ', '4.197227442768982']
['  Industrials  ', '3.9404631973390454']
['Consumer Discretionary', '3.4998512756938203']
['  Financials   ', '3.3456772677837137']
['Consumer Staples', '2.7012396022606118']
['Information Technology', '2.298128249883806']
['   Utilities   ', '2.190323941203284']
['Telecommunications Services', '1.70501159249443']
['   Materials   ', '0.9834609384066598']
['    Energy     ', '-1.2573757842416295']




A value greater than 1 is considered acceptable, therefore a resonable portfolio would include stocks in:
<ul>
    <li>Real Estate</li>
    <li>Health Care</li>
    <li>Industrials</li>
    <li>Consumer Discretionary</li>
    <li>Financials</li>
    <li>Consumer Staples</li>
    <li>Information Technology</li>
    <li>Utilities</li>
    <li>Telecommunications Services</li>
</ul>

<h2>STRATEGIES COMPARISON</h2>

In this section we implement three (very simple) trading models with comparative results to check which one it's the best performing; we consider three strategies:
<ul>
    <li>Overall buy-and-hold: buying shares for a stock at the Open price on the first trading day (2010-01-04) and selling them at the Close price of the last trading day.</li>
    <li>Monthly buy-and-hold: buying shares for a stock at the Open price on the first trading day of each month and selling them at the Close price of the last trading day of the same month.</li>
    <li>Conditional buy-and-hold: buying shares for a stock at the Open price on the first trading day of a month and selling them at the Close price of the last trading day of the same month IF there's an "upward trend", i.e. a 10-day moving average at the beginning of the month is higher than a 30-day moving average.</li>
</ul>
We'll compute the results both per share and multiplied by a lot size, focus on the top 10 shares in terms if closing price:

In [12]:
sql = """
SELECT symbol as top_symbol, SUM(close) as Total
FROM price_uniformed
GROUP BY symbol
ORDER BY Total DESC
LIMIT 10
;
"""

cur = conn.cursor()
cur.execute(sql)
rows = cur.fetchall()
cnames = list(map(lambda x: x[0], cur.description))
cur.close()
format_table(cnames, rows)

['  top_symbol   ', '     Total     ']
----------------------------------------
['     PCLN      ', '1521411.018718999']
['     GOOGL     ', '1190377.8586189984']
['     GOOG      ', '1176544.1359700002']
['      AZO      ', ' 826391.649805 ']
['     ISRG      ', '813999.7806260004']
['      CMG      ', ' 724629.530008 ']
['     AMZN      ', '595378.1406979997']
['     AAPL      ', '551377.4896539997']
['      MA       ', '472034.4399309996']
['      BLK      ', '461275.46024000045']




The top stocks for closing prices are:
<ul>
    <li>PCLN (Now Booking Holdings Inc.)</li>
    <li>AAPL (Apple Inc.)</li>
    <li>GOOGL (Alphabet Inc.)</li>
    <li>ISRG (Intuitive Surgical Inc.)</li>
    <li>GOOG (Alphabet Inc.)</li>
    <li>CMG (Chipotle Mexican Grill Inc.)</li>
    <li>AMZN (Amazon.com Inc.)</li>
    <li>BLK (BlackRock Inc.)</li>
    <li>MA (Mastercard Inc.)</li>
    <li>MTD (Mettler-Toledo International Inc.)</li>
</ul>

In [13]:
sql_results = """
/* Creating a table holding the summary results */
DROP TABLE IF EXISTS strategies_results;
CREATE TABLE IF NOT EXISTS strategies_results(
symbol text,
lot_size integer,
G_L_share_hold real,
G_L_lot_hold real,
G_L_share_monthly real,
G_L_lot_monthly real,
G_L_share_mav real,
G_L_lot_mav real
);
"""


sql_smd = """
/* Creating a temporary table with all the data necessary for the three trading rules 
(first day Open price and last day Close price for each month, Monving Averages values 
for the first day of each month) */

CREATE TEMPORARY TABLE IF NOT EXISTS symbol_monthly_data AS SELECT * FROM
        (
        WITH first_day AS
        (SELECT * FROM 
        (SELECT symbol,
            date,
            Open,
            CAST(substr(date, 1, 4) AS INTEGER) AS year,
            CAST(substr(date, 6, 2) AS INTEGER) AS month,
            AVG(Close) OVER (
                ORDER BY date
                ROWS BETWEEN 9 PRECEDING AND CURRENT ROW
                ) mav_10, 
            AVG(Close) OVER (
                ORDER BY date
                ROWS BETWEEN 29 PRECEDING AND CURRENT ROW
                ) mav_30
            FROM price_uniformed
            WHERE symbol = :symbol)
        GROUP BY year, month
        HAVING date = MIN(date)),
        
        last_day AS
        (SELECT symbol, year, month, Close FROM 
        (SELECT symbol,
            date,
            CAST(substr(date, 1, 4) AS INTEGER) AS year,
            CAST(substr(date, 6, 2) AS INTEGER) AS month,
            Close
            FROM price_uniformed
            WHERE symbol = :symbol)
        GROUP BY year, month
        HAVING date = MAX(date))
        SELECT first_day.symbol, first_day.year, first_day.month, first_day.Open, first_day.mav_10, first_day.mav_30, last_day.Close
        FROM first_day INNER JOIN last_day ON (first_day.year = last_day.year AND first_day.month = last_day.month)
        );
        """

sql_insert = """
/* Populating the results table with data for each share and the entire lot */

INSERT INTO strategies_results SELECT * FROM (
WITH fo AS 
(SELECT Open AS First_Open 
FROM symbol_monthly_data WHERE 
(year = (SELECT MIN(year) FROM symbol_monthly_data) AND month = (SELECT MIN(month) FROM symbol_monthly_data))
),
lc AS
(SELECT Close AS Last_Close 
FROM symbol_monthly_data WHERE 
(year = (SELECT MAX(year) FROM symbol_monthly_data) AND month = (SELECT MAX(month) FROM symbol_monthly_data))
),
monthly AS
(SELECT
SUM(Close - Open) AS monthly_share,
SUM(Close - Open) * :lot_size AS monthly_lot
FROM symbol_monthly_data),
mav AS
(SELECT
SUM(Close - Open) AS mav_share,
SUM(Close - Open) * :lot_size AS mav_lot
FROM symbol_monthly_data WHERE mav_10 > mav_30)
SELECT DISTINCT
symbol_monthly_data.symbol,
:lot_size,
lc.Last_Close - fo.First_Open,
(lc.Last_Close - fo.First_Open) * :lot_size,
monthly.monthly_share,
monthly.monthly_lot,
mav.mav_share,
mav.mav_lot
FROM symbol_monthly_data, fo, lc, monthly, mav
);
"""

sql_clean = """
        /* Dropping the temporary table when all the results are computed */
        DROP TABLE IF EXISTS temp.symbol_monthly_data;
        """

We'll compute the results using the same lot size for all stocks:

In [14]:
top10_symbols = ['PCLN', 'GOOGL', 'GOOG', 'AZO', 'ISRG', 'CMG', 'AMZN', 'AAPL', 'MA', 'BLK']
lot_sizes = [150 for _ in range(len(top10_symbols))]
cur = conn.cursor()
cur.executescript(sql_results)
cur.executescript(sql_clean)
conn.commit()
cur.close()
for symbol, lot_size in zip(top10_symbols, lot_sizes):
    params_dict = {"symbol" : symbol, "lot_size" : lot_size}
    cur = conn.cursor()
    cur.execute(sql_smd, params_dict)
    conn.commit()
    cur.execute(sql_insert, params_dict)
    conn.commit()
    cur.executescript(sql_clean)
    conn.commit()
    cur.close()
print('Summary results produced!')

Summary results produced!


Finally we show the results for the whole lot size for the three different strategies:

In [15]:
sql_results = """
SELECT 
symbol,
lot_size,
ROUND(G_L_lot_hold, 2) AS G_L_lot_hold,
ROUND(G_L_lot_monthly, 2) AS G_L_lot_monthly,
ROUND(G_L_lot_mav, 2) AS G_L_lot_mav
FROM strategies_results;
"""

cur = conn.cursor()
cur.execute(sql_results)
rows = cur.fetchall()
cnames = list(map(lambda x : x[0], cur.description))
cur.close()
format_table(cnames, rows, rows_limit = 11)

['    symbol     ', '   lot_size    ', ' G_L_lot_hold  ', 'G_L_lot_monthly', '  G_L_lot_mav  ']
----------------------------------------------------------------------------------------------------
['     PCLN      ', '      150      ', '   186561.01   ', '   166999.47   ', '   30511.54    ']
['     GOOGL     ', '      150      ', '    24825.0    ', '    14125.5    ', '   39262.46    ']
['     GOOG      ', '      150      ', '   21730.34    ', '   12509.76    ', '   -55786.39   ']
['      AZO      ', '      150      ', '    94620.0    ', '   89667.02    ', '    40558.5    ']
['     ISRG      ', '      150      ', '    49045.5    ', '   28603.49    ', '    9063.01    ']
['      CMG      ', '      150      ', '    43158.0    ', '   42637.51    ', '   26467.53    ']
['     AMZN      ', '      150      ', '    92043.0    ', '   92245.53    ', '   65389.52    ']
['     AAPL      ', '      150      ', '   -14641.5    ', '   -30991.5    ', '   -34755.0    ']
['      MA       ', '      150     

In this case the overall buy-and-hold rule outperforms the other two for almost all the stocks; it's worth stressing however that these trading rules are overly simplistic and to be considered as benchmarks rather than recommended strategies.

In [16]:
#Cleaning remaining views

sql = """
DROP VIEW IF EXISTS symbols_uniformed;
DROP VIEW IF EXISTS symbol_sector;
"""

cur = conn.cursor()
cur.executescript(sql)
cur.close()